🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Author : Laurent Sorba*

*Date : 04/07/2025*

**Description :**

This notebook filters and extracts public transportation data (GTFS format) according to a department number. In our case specifically for the Isère department (code 38) in France.
The notebook downloads GTFS (General Transit Feed Specification) data from various transportation providers and uses precise administrative boundaries to spatially filter bus stops and routes that operate within the department boundaries.

See https://github.com/data-for-good-grenoble/mobilite_durable/issues/13

## Définition des fonctions

In [ ]:
import io
import json
import zipfile
from pathlib import Path

import geopandas as gpd
import pandas as pd
import requests
from shapely.geometry import Point


# 1. Télécharger et extraire les données GTFS
def download_gtfs_data(gtfs_url):
    """Télécharge et extrait les données GTFS"""
    print(f"=== Télécharge les données GTFS depuis {gtfs_url} ===")
    response = requests.get(gtfs_url)
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
        # Extraire les fichiers nécessaires
        stops_df = pd.read_csv(zip_file.open("stops.txt"))
        print(f"Chargé {len(stops_df)} arrêts")
        routes_df = pd.read_csv(zip_file.open("routes.txt"))
        print(f"Chargé {len(routes_df)} lignes")

        # Autres fichiers GTFS si nécessaire
        try:
            stop_times_df = pd.read_csv(zip_file.open("stop_times.txt"))
            print(f"(Chargé {len(routes_df)} stop_times)")
            trips_df = pd.read_csv(zip_file.open("trips.txt"))
            print(f"(Chargé {len(routes_df)} trips)")
        except KeyError:
            stop_times_df = None
            trips_df = None

    return stops_df, routes_df, stop_times_df, trips_df


# 2. Filtrer les arrêts par département de l'Isère par défaut
def filter_stops_by_department(stops_df, department_code="38"):
    """Filtre les arrêts par code département"""
    return filter_with_administrative_boundaries(stops_df, department_code)


# 3. Filtrer les lignes de transport correspondant
def filter_routes_by_stops(routes_df, filtered_stops, trips_df=None, stop_times_df=None):
    """Filtre les lignes qui desservent les arrêts filtrés"""
    if trips_df is not None and stop_times_df is not None:
        # Trouver les trips qui passent par les arrêts filtrés
        department_stop_ids = filtered_stops["stop_id"].unique()
        trips_with_department_stops = stop_times_df[
            stop_times_df["stop_id"].isin(department_stop_ids)
        ]["trip_id"].unique()

        # Trouver les routes correspondantes
        routes_in_department = trips_df[trips_df["trip_id"].isin(trips_with_department_stops)][
            "route_id"
        ].unique()

        filtered_routes = routes_df[routes_df["route_id"].isin(routes_in_department)]
    else:
        # Si pas de données de correspondance, retourner toutes les routes
        filtered_routes = routes_df

    return filtered_routes


def filter_with_administrative_boundaries(stops_df, department_code):
    """
    Filtre les arrêts de transport en utilisant les limites administratives précises
    du département de l'Isère
    """

    # 1. Charger les limites administratives du département
    department_boundary = download_department_boundary(department_code)

    if department_boundary is None:
        print(
            "Impossible de télécharger les limites administratives. Utilisation du filtrage par coordonnées."
        )
        exit

    # 2. Convertir les arrêts en GeoDataFrame
    stops_gdf = create_stops_geodataframe(stops_df)

    # 3. Filtrer les arrêts qui se trouvent dans le département
    filtered_stops = spatial_filter_stops(stops_gdf, department_boundary)

    return filtered_stops


def download_department_boundary(department_code="38"):
    """
    Charge ou télécharge les limites administratives du département de l'Isère
    """

    # Utiliser les données de data.gouv.fr
    try:
        boundary = load_department_geometry(department_code)
        if boundary is not None:
            return boundary
    except Exception as e:
        print(f"Erreur avec data.gouv.fr: {e}")

    # Fallback : Utiliser des coordonnées prédéfinies
    try:
        boundary = create_isere_boundary_from_coords()
        return boundary
    except Exception as e:
        print(f"Erreur avec coordonnées prédéfinies: {e}")

    return None


def download_from_data_gouv(file_path):
    """
    Télécharge depuis data.gouv.fr - Contours des départements français
    """
    # URL des contours des départements français
    url = "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb"
    print(f"=== Télécharge les contours des départements français {url} ===")
    try:
        # Télécharger le GeoJSON
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Récupérer le contenu JSON
        geojson_data = response.json()

        # Créer le répertoire s'il n'existe pas
        file_path = Path(file_path)
        file_path.parent.mkdir(parents=True, exist_ok=True)

        # Sauvegarder dans le fichier
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(geojson_data, f, ensure_ascii=False, indent=2)

        print(f"Fichier sauvegardé avec succès : {file_path}")
        print(f"Nombre de départements : {len(geojson_data['features'])}")

    except Exception as e:
        print(f"Erreur lors du téléchargement depuis data.gouv.fr: {e}")
        return None


def load_department_geometry(department_code="38"):
    """
    Télécharge depuis data.gouv.fr - Contours des départements français
    """
    # URL des contours des départements français
    file = "../data/transportdatagouv/contour-des-departements.geojson"
    print(f"=== Charge les contours des départements français depuis le fichier {file} ===")

    try:
        # Test if file exists
        if not Path(file).is_file():
            print(f"-> Le fichier n'existe pas, téléchargement nécessaire.")
            download_from_data_gouv(file)

        # Charger le GeoJSON
        france_departments = gpd.read_file(file)

        # Filtrer sur un département
        department_boundary = france_departments[france_departments["code"] == department_code]

        if not department_boundary.empty:
            return department_boundary.iloc[0].geometry
        else:
            print(f"Département {department_code} non trouvé dans les données")
            return None

    except Exception as e:
        print(f"Erreur lors du chargement des contours des départements: {e}")
        return None


def create_isere_boundary_from_coords():
    """
    Crée une approximation des limites de l'Isère à partir de coordonnées connues
    """
    from shapely.geometry import Polygon

    print(f"=== Fallback: Utiliser des coordonnées prédéfinies de l'Isère ===")

    # Coordonnées approximatives des limites de l'Isère
    isere_coords = [
        (5.2, 44.8),  # Sud-Ouest
        (6.3, 44.8),  # Sud-Est
        (6.3, 45.9),  # Nord-Est
        (5.2, 45.9),  # Nord-Ouest
        (5.2, 44.8),  # Fermeture du polygone
    ]

    boundary = Polygon(isere_coords)
    return boundary


def create_stops_geodataframe(stops_df):
    """
    Convertit le DataFrame des arrêts en GeoDataFrame
    """
    # Créer des objets Point à partir des coordonnées
    geometry = [Point(xy) for xy in zip(stops_df["stop_lon"], stops_df["stop_lat"])]

    # Créer le GeoDataFrame
    stops_gdf = gpd.GeoDataFrame(
        stops_df,
        geometry=geometry,
        crs="EPSG:4326",  # WGS84
    )

    return stops_gdf


def spatial_filter_stops(stops_gdf, department_boundary):
    """
    Filtre spatialement les arrêts qui se trouvent dans les limites du département
    """
    print(
        f"=== Filtre spatialement les arrêts qui se trouvent dans les limites du département ==="
    )
    # Créer un GeoDataFrame pour la limite du département
    if hasattr(department_boundary, "crs"):
        boundary_gdf = gpd.GeoDataFrame(
            [1], geometry=[department_boundary], crs=department_boundary.crs
        )
    else:
        boundary_gdf = gpd.GeoDataFrame([1], geometry=[department_boundary], crs="EPSG:4326")

    # S'assurer que les deux GeoDataFrames ont le même CRS
    if stops_gdf.crs != boundary_gdf.crs:
        stops_gdf = stops_gdf.to_crs(boundary_gdf.crs)

    # Filtrage spatial : garder les arrêts qui intersectent avec la limite
    filtered_stops = gpd.sjoin(stops_gdf, boundary_gdf, how="inner", predicate="within")

    # Supprimer les colonnes ajoutées par sjoin
    filtered_stops = filtered_stops.drop(columns=["index_right"], errors="ignore")

    return filtered_stops

## Définition des variables

In [ ]:
pd.set_option("display.max_columns", None)

# URLs des données GTFS contenant l'Isère
gtfs_sources = {
    # Definition https://transport.data.gouv.fr/datasets/agregat-oura
    "agregat-oura": "https://api.oura3.cityway.fr/dataflow/offre-tc/download?provider=OURA&dataFormat=GTFS&dataProfil=OPENDATA",
    # Definition https://transport.data.gouv.fr/datasets/reseau-cars-region-isere-38
    "reseau-cars-region-isere-38": "https://www.itinisere.fr/fr/donnees-open-data/169/OpenData/Download?fileName=CG38.GTFS.zip",
    # Definition https://transport.data.gouv.fr/datasets/horaires-theoriques-du-reseau-tag
    "TAG_Grenoble": "https://data.mobilites-m.fr/api/gtfs/SEM",
}

department_code = "38"
data_set = "agregat-oura"
gtfs_url = gtfs_sources[data_set]

In [ ]:
print(f"=== Filtrage avec limites administratives du '{department_code}' sur '{data_set}' ===")

### Télécharger les données GTFS

In [ ]:
stops_df, routes_df, stop_times_df, trips_df = download_gtfs_data(gtfs_url)

### Filtrer les arrêts

In [ ]:
filtered_stops = filter_stops_by_department(stops_df, department_code)
print(
    f"Nombre d'arrêts dans le {department_code}: {len(filtered_stops)} sur un total de {len(stops_df)}"
)

### Filtrer les lignes

In [ ]:
filtered_routes = filter_routes_by_stops(routes_df, filtered_stops, trips_df, stop_times_df)
print(
    f"Nombre de lignes dans le {department_code}: {len(filtered_routes)} sur un total de {len(routes_df)}"
)

## Résultats

In [ ]:
# Exemples d'arrêts
print("\nPremiers arrêts filtrés:")
print(filtered_stops[["stop_id", "stop_code", "stop_name", "stop_lat", "stop_lon"]].head(10))

In [ ]:
# Exemples de lignes
print("\nPremières lignes filtrées:")
print(filtered_routes[["route_id", "route_short_name", "route_long_name"]].head(10))